# 상태유지 스택 순환신경망 모델

In [3]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY=[], []
    for i in range(len(signal_data)-look_back):
        dataX.append(signal_data[i:(i+look_back),0])
        dataY.append(signal_data[i+look_back,0])
    return np.array(dataX), np.array(dataY)

In [5]:
class CustomHistory(keras.callbacks.Callback):
    def init(self):
        self.train_loss=[]
        self.val_loss=[]
    
    def on_epoch_end(self, batch, logs={}):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))
    

In [6]:
look_back=20

signal_data=np.cos(np.arange(1600)*(20.*np.pi/1000))[:,None]


In [7]:
scaler=MinMaxScaler(feature_range=(0,1))
signal_data=scaler.fit_transform(signal_data)

In [8]:
train = signal_data[0:800]
val = signal_data[800:1200]
test = signal_data[1200:]

In [9]:
x_train, y_train = create_dataset(train, look_back)
x_val, y_val = create_dataset(val, look_back)
x_test, y_test = create_dataset(test, look_back)

In [ ]:
x_train=np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_val=np.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))
x_test=np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
model= Sequential()
for i in range(2):
    model.add(LSTM(64, batch_input_shape=(1, look_back,1) ,stateful=True, return_sequences=True))
    model.add(Dropout(0.3))

model.add(LSTM(32,  batch_input_shape=(1, look_back,1) , stateful=True))
model.add(Dropout(0.3))
model.add(Dense(1))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='MSE',optimizer='adam')
custom_hist=CustomHistory()
custom_hist.init()

In [ ]:
for i in range(200):
    print(i)
    model.fit( x_train, y_train, epochs=1, batch_size=1, shuffle=False, callbacks=[custom_hist], validation_data=(x_val, y_val))
    model.reset_states()

In [ ]:
trainScore=model.evaluate(x_train, y_train, verbose=0,batch_size=1)
model.reset_states()
print('Train Score:', trainScore)

ValScore=model.evaluate(x_val, y_val, verbose=0,batch_size=1)
model.reset_states()
print('Train Score:', ValScore)

testScore=model.evaluate(x_test, y_test, verbose=0,batch_size=1)
model.reset_states()
print('Train Score:', testScore)


In [ ]:
look_ahead=350
xhat=x_test[0]
predictions=np.zeros((look_ahead,1))
for i in range(look_ahead):
    prediction=model.predict(np.array([xhat]), batch_size=1)
    predictions[i]=prediction
    xhat=np.vstack([xhat[1:,],prediction])

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(np.arange(look_ahead),predictions,'r',label='prediction')
plt.plot(np.arange(look_ahead),y_test[:look_ahead],label='test function')
plt.legend()
plt.show()
